#web scraping thingy 

In [133]:
# ["type", "beds", "price", "location", "url", "aquisition", "contact", "duration", "furnished", "serviced", "imageurl"] //compare rent vs buy(month v day), should I buy/recommeded
# paginationSelector = soup.find('div', class_='pagination-area').nav.ul.findAll('li')[3].a['href']
# paginationSelector

In [5]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from bs4.element import Comment
import re
import csv
import json
import pandas as pd

In [2]:

def getPageData(url):
    property =  0

    req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    soup = BeautifulSoup(webpage, 'html.parser')   
    listing_property = soup.find_all("div", class_='listings-property')
    data = []

    for i in listing_property:
        if "sponsored-listing" in i["class"]:
            listing_property.remove(i)
        else:
            property = listing_property.index(i)
            amenities = listing_property[property].find('div', class_='fur-areea').findAll("span", recursive=False)
            beds = amenities[0].text[0:1]
            bathrooms = amenities[1].text[0:1]
            toilets = amenities[2].text[0:1]
            if beds == "0" or beds == 0 or beds == "":
                break
            if bathrooms == "0" or bathrooms == 0 or bathrooms == "":
                break
            if toilets == "0" or toilets == 0 or toilets == "":
                break
            agentContact = re.sub("[\n ]", "", listing_property[property].find('div', class_='phone-icon').text)
            listingTitile = listing_property[property].find('h3', class_='listings-property-title2').text
            listingURL = "https://www.propertypro.ng" + listing_property[property].find("div", "result-img").a['href']
            price = ''.join(re.findall(r'\d+', listing_property[property].find('h3', class_='listings-price').text))
            location = listing_property[property].find('div', class_='result-img').a.img['alt'].split(' ')
            street = " ".join(listing_property[property].find('div', class_='single-room-text').findAll("h4")[1].text.split(' ')[ : -2])
            if street == "":
                street = "unstated"
            city = location[-2]
            state = location[-1]
            try:
                status = str(re.sub("[\n]", "", listing_property[4].find('div', class_='furnished-btn').text))
                if status == "":
                    status = "old"
            except: 
                status = "old"
            try:
                dateUpdated = " ".join(listing_property[property].find('div', class_='single-room-text').find("h5").text.split()[-3 :])
            except:
                dateUpdated = "not stated"
            try:
                imagesProvided =  re.sub("[\n ]", "", listing_property[property].find('div', class_='room-icons').text)
            except:
                imagesProvided = 3 
            data.append([listingTitile, price, bathrooms, beds, toilets, street, city, state, status, agentContact, imagesProvided, listingURL, dateUpdated])
    return data


In [ ]:
numIterations = 300
startingUrl = "https://www.propertypro.ng/property-for-rent/house/?search=&auto=&bedroom=&min_price=&max_price="
data = []

for pageNum  in range(numIterations):
    if pageNum == 0:
        url = startingUrl
    else:
        url =  startingUrl + "&page=" + str(pageNum)
    try: 
        searchResult = getPageData(url)
        data = data + searchResult
        print(pageNum)
    except Exception as e:
        print(pageNum, e)

In [4]:
# WRITE DATA TO CSV FILE
header =["listingTitile", "price", "bathrooms", "beds", "toilets", "street", "city", "state", "status", "agentContact", "imagesProvided", "listingURL", "dateUpdated"]
# header = [listingTitile"," price"," bathrooms"," beds", "toilets", "street", "city", "state", "status", "agentContact", "imagesProvided", "listingURL", "dateUpdated"]
with open('HouseRent6.csv', 'w', encoding='UTF8') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(header)
    for i in data:
        writer.writerow(i)

In [6]:
propertyForSale = pd.read_csv("ScrapedSalePropertyDataSet.csv")
propertyForRent = pd.read_csv("ScrapedRentalDataSet.csv")

In [14]:
req = Request("https://www.propertypro.ng/property-for-sale?search=&auto=&type=&bedroom=&min_price=&max_price=" , headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
soup = BeautifulSoup(webpage, 'html.parser')   
listing_property = soup.find_all("div", class_='listings-property')
for i in listing_property:
    if "sponsored-listing" in i["class"]:
        listing_property.remove(i)

In [6]:
# CREATE DATAFRAME
header =["listingTitile", "price", "bathrooms", "beds", "toilets", "street", "city", "state", "status", "agentContact", "imagesProvided", "listingURL", "dateUpdated"]
df = pd.DataFrame(data=data, columns=header)
df.to_json (r'propertyproData' , orient='records',)
df.head()

,listingTitile,price,bathrooms,beds,toilets,street,city,state,status,agentContact,imagesProvided,listingURL,dateUpdated
0,Newly Renovated & Spacious 4bedoom Duplex (no ...,5000000,5,4,5,Adeniran Ogunsanya,Surulere,Lagos,Newly Built,08167742895,20,https://www.propertypro.ng/property/4-bedroom-...,12 Jul 2022
1,4 Bedroom Semi Detached For Rent,6500000,5,4,5,Osapa London,Lekki,Lagos,Newly Built,09035137760,9,https://www.propertypro.ng/property/4-bedroom-...,12 Jul 2022
2,Newly Built 4 Bedroom Terrace Duplex,6000000,4,4,5,Lekki Phase 1,Lekki,Lagos,Newly Built,07047998445,9,https://www.propertypro.ng/property/4-bedroom-...,05 May 2022
3,4 Bedroom Semi Detached Duplex,6500000,5,4,5,Ologolo,Lekki,Lagos,Newly Built,09035137760,10,https://www.propertypro.ng/property/4-bedroom-...,11 Jul 2022
4,4 Bedroom Terrace Apartment,6000000,5,4,5,Spar Road Just By House On The Rock And Nicon ...,Lekki,Lagos,Newly Built,09035137760,9,https://www.propertypro.ng/property/4-bedroom-...,11 Jul 2022


In [1]:
phone_array = [8037939912, 8037939912, 8038024880, 8038123030, 8038125965, 8038125965, 8038359282, 8038359282, 8038389084, 8038479985, 8038592381, 8039220606, 8050383215, 8050494374, 8050494374, 8053979149, 8053979149, 8053979149, 8053979149, 8053979149, 8053979149, 8053979149, 8053979149, 8053979149, 8054797512, 8055652402, 8056267437, 8056267437, 8056267437, 8056267437, 8056267437, 8056267437, 8057022131, 8057022131, 8059149400, 8060140819, 8060283193, 8060283193, 8060283193, 8060623971, 8061641323, 8061641323, 8061987440, 8061987440, 8061987440, 8061987440, 8061987440, 8061987440, 8062345097, 8062345097, 8062345097, 8062345097, 8062345097, 8062391186, 8062529809, 8062529809, 8062529809, 8062881764, 8063768619, 8064323103, 8064537923, 8065305104, 8065305104, 8065329702, 8065558146, 8065795091, 8065795091, 8065795091, 8065795091, 8065795091, 8065795091, 8065795091, 8066392132, 8066627552, 8066627552, 8066627552, 8066627552, 8067081484, 8067081484, 8067081484, 8067081484, 8067081484, 8067741227, 8067992859, 8067992859, 8068361629, 8069566040, 8069566040, 8072800567, 8075711336, 8076508977, 8076508977, 8076508977, 8076508977, 8076508977, 8077738880, 8077738880, 8080564797, 8081501656, 8082163583, 8082163583, 8082163583, 8082163583, 8082163583, 8082163583, 8082163583, 8082163583, 8082163583, 8082163583, 8082163583, 8082163583, 8082163583, 8082163583, 8082163583, 8082163583, 8082163583, 8082163583, 8082163583, 8082163583, 8082163583, 8082163583, 8082163583, 8082163583, 8082163583, 8082163583, 8082163583, 8082163583, 8082163583, 8082163583, 8082163583, 8082163583, 8082163583, 8082163583, 8082163583, 8082163583, 8082163583, 8082163583, 8082163583, 8082163583, 8082163583, 8082163583, 8082163583, 8082163583, 8082163583, 8082163583, 8082163583, 8082163583, 8082821053, 8083241498, 8086267650, 8089633335, 8089633335, 8089633335, 8093708615, 8093708615, 8094163102, 8094163102, 8095996837, 8097100871, 8099275848, 8099275848, 8099275848, 8100073388, 8100073388, 8100073388, 8100073388, 8100694574, 8100694574, 8104717713, 8105605371, 8106600637, 8106600637, 8106600637, 8106600637, 8106600637, 8106600637, 8106600637, 8106600637, 8106600637, 8106600637, 8106600637, 8106600637, 8106600637, 8106600637, 8106600637, 8106600637, 8106600637, 8106600637, 8106600637, 8111113467, 8111113467, 8111113467, 8111113467, 8111888039, 8112095650, 8114892643, 8114892643, 8114892643, 8114892643, 8114892643, 8114892643, 8114892643, 8114892643, 8115380597, 8117248757, 8117340933, 8117340933, 8117340933, 8117340933, 8117340933, 8117340933, 8117340933, 8117340933, 8117340933, 8117340933, 8119470091, 8120765101, 8120765101, 8121377323, 8121377323, 8122986390, 8124031574, 8124031574, 8124031574, 8124031574, 8124269637, 8124269637, 8124269637, 8124269637, 8124269637, 8124269637, 8124269637, 8124269637, 8124269637, 8124269637, 8124269637, 8124269637, 8124269637, 8124269637, 8124269637, 8124269637, 8124269637, 8124269637, 8124269637, 8124269637, 8124269637, 8124269637, 8124269637, 8124269637, 8124269637, 8124269637, 8124269637, 8124269637, 8124269637, 8124269637, 8124269637, 8124269637, 8124269637, 8124269637, 8124269637, 8124269637, 8126759308, 8130263266, 8130986430, 8131315389, 8131315389, 8132415332, 8133154602, 8133194443, 8133194443, 8133194443, 8133194443, 8133194443, 8133419582, 8136670481, 8136670481, 8136670481, 8136670481, 8137377290, 8137377290, 8137377290, 8137584753, 8138248835, 8138310553, 8138800178, 8138800178, 8138800178, 8138805351, 8138805351, 8139265236, 8139265236, 8139265236, 8139265236, 8139265236, 8139265236, 8139265236, 8139265236, 8139635919, 8139635919, 8139635919, 8142223792, 8143129775, 8143129775, 8143129775, 8143129775, 8143129775, 8143129775, 8143129775, 8143129775, 8143129775, 8147562407, 8147562407, 8147562407, 8147562407, 8147562407, 8147747114, 8147747114, 8147747114, 8147747114, 8147747114, 8147747114, 8147747114, 8147747114, 8148211572, 8148211572, 8148211572, 8148211572, 8148495671, 8148495671, 8148495671, 8148495671, 8148495671, 8148495671, 8148495671, 8148495671, 8148495671, 8148495671, 8149249272, 8153985682, 8155321626, 8160442385, 8160442385, 8160442385, 8160442385, 8160442385, 8160442385, 8160442385, 8160772746, 8163569554, 8163569554, 8164687732, 8164687732, 8164687732, 8164687732, 8164687732, 8164687732, 8164687732, 8164687732, 8164968133, 8164968133, 8164968133, 8164968133, 8164968133, 8164968133, 8165277357, 8166553629, 8166745180, 8166851351, 8167383898, 8167383898, 8167383898, 8167383898, 8167383898, 8167742895, 8167742895, 8167742895, 8167742895, 8167742895, 8167978798, 8168064058, 8168064058, 8168064058, 8168064058, 8168064058, 8168064058, 8168064058, 8168064058, 8168064058, 8168064058, 8168064058, 8169790456, 8169790456, 8177400441, 8177400441, 8177400441, 8177400441, 8177400441, 8177766115, 8177766115, 8177766115, 8177766550, 8177766550, 8180249383, 8181251091, 8181251091, 8182073205, 8182073205, 8182073205, 8182911042, 8182911042, 8182911042, 8184412041, 8184412041, 8185551251, 8185551251, 8185551251, 8185551251, 8185551251, 8185551251, 8185551251, 8185551251, 8186969203, 8186969203, 8186969203, 9013619321, 9013619321, 9014004443, 9019001143, 9019001143, 9019001143, 9019001143, 9023730900, 9023730900, 9023730900, 9023730900, 9023730900, 9023730900, 9026318426, 9026318426, 9026318426, 9026318426, 9026318426, 9026318426, 9026318426, 9026318426, 9026318426, 9029604948, 9029604948, 9029604948, 9029604948, 9029604948, 9030097979, 9034504273, 9034504273, 9034504273, 9034504273, 9034504273, 9034504273, 9035137760, 9035137760, 9035137760, 9035137760, 9035137760, 9035137760, 9035137760, 9035137760, 9035137760, 9035137760, 9035137760, 9035137760, 9035137760, 9035137760, 9035137760, 9035137760, 9035137760, 9035137760, 9035137760, 9035137760, 9035137760, 9035137760, 9035137760, 9035137760, 9035137760, 9035137760, 9035137760, 9035137760, 9035137760, 9035137760, 9035137760, 9035137760, 9035137760, 9035137760, 9035137760, 9035137760, 9035137760, 9035137760, 9035137760, 9035137760, 9035137760, 9035137760, 9035137760, 9035137760, 9035137760, 9035137760, 9035137760, 9035137760, 9041150803, 9041150803, 9041150803, 9041150803, 9041150803, 9041150803, 9044442244, 9045982770, 9045982770, 9045982770, 9045982770, 9045982770, 9045982770, 9045982770, 9050518082, 9050518082, 9050518082, 9050857650, 9050857650, 9053588424, 9053588424, 9053588424, 9053588424, 9053588424, 9053588424, 9053588424, 9053588424, 9053588424, 9055294872, 9055794120, 9055794120, 9055794120, 9056267480, 9056267480, 9056267480, 9056267480, 9056267480, 9056267480, 9056267480, 9058645473, 9061500067, 9061623299, 9061623299, 9061623299, 9061623299, 9064090448, 9064090448, 9067320607, 9067320607, 9067320607, 9067320607, 9067320607, 9067320607, 9069656035, 9070732024, 9070732024, 9071394259, 9071394259, 9071394259, 9071767938, 9071767938, 9074162101, 9075318146, 9079005774, 9079005774, 9080088779, 9080088779, 9080088779, 9080088779, 9080088779, 9080088779, 9080088779, 9081104133, 9083616685, 9085096183, 9085096183, 9085096183, 9087744002, 9090090450, 9090090450, 9090407416, 9096807820, 9098077985, 9098077985, 9099980793, 9121180993, 9121180993, 9121180993, 9121180993, 9124604075, 9124604075, 9131636829, 9168011083, 9168011083, 9168011083, 9168011083, 9168011083, 9168011083, 9168011083, 9168011083, 9168011083, 9168011083, 9168011083, 90902999998, 90902999998]

In [2]:
phone_array2 = [ '0' + str( item) for item in phone_array ] # remove empty strings from the list

In [8]:
phone_array2

print(len (phone_array2))


# unique values in the list

unique_phone_array = list( set (phone_array2))

611


In [13]:
unique_phone_array


with open("output.txt", "w") as txt_file:
    for line in unique_phone_array:
        txt_file.write(" ".join(line) + "\n") # works with any number of elements

